# Cómo crear un Agente LLM sencillo con LangGraph
* Agente LLM con memoria que utiliza herramientas muy básicas.

Los modelos de lenguaje no pueden hacer nada por sí solos; solo pueden crear texto en función de lo que se les pida. Sin embargo, LangChain permite a las personas crear agentes (piense en ellos como sistemas inteligentes o ayudantes) que utilizan modelos de lenguaje para pensar y decidir qué hacer a continuación.

Así es como funciona:
1. **Utilice el modelo de lenguaje como cerebro**: el agente utiliza el modelo de lenguaje para determinar qué acciones debe realizar, en función de la información que tiene o de lo que se le pida que haga.
2. **Actuación**: después de tomar una decisión, el agente procede a realizar esas acciones.
3. **Aprendizaje y ajuste**: una vez realizadas las acciones, los resultados se pueden devolver al modelo de lenguaje. Esto ayuda al modelo a verificar si todo está completo o si necesita hacer algo más.

Básicamente, LangChain ayuda a convertir un modelo de lenguaje de una herramienta para escribir y responder a un sistema que puede actuar y reaccionar, casi como un cerebro robótico muy simple.

Aquí crearemos un agente que pueda interactuar con un motor de búsqueda. Podrás hacerle preguntas a este agente, observarlo mientras llama a la herramienta de búsqueda y tener conversaciones con él.

In [ ]:
import warnings
import os

from langchain._api import LangChainDeprecationWarning
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

# Creamos el modelo a utilizar
chatModel = ChatOpenAI(model="gpt-4o-mini")

# Creamos nuestro formateador de salida
output_parser = StrOutputParser()

## Agentes
* Los agentes utilizan los LLM como motores de razonamiento para determinar qué acciones tomar.

## Agente que utiliza herramientas
* Para este agente básico, utilizaremos solo una herramienta. En los próximos proyectos avanzados, aprenderá a utilizar agentes con varias herramientas.
* **Nuestra herramienta de elección será Tavily**, un motor de búsqueda.
#### Clave API de Tavily
* Deberá agregar su clave API de Tavily en el archivo .env. Obtenga una clave registrándose en su [sitio web](https://tavily.com/).

Si está utilizando el shell de poetry precargado, no necesita instalar el siguiente paquete porque ya está precargado para usted, sino deberá instalarlo con:

`#pip install langchain-community`

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("¿Quién fue la estrella de la Eurocopa en 2024?")

In [ ]:
tools = [search]

* **Para permitir que este modelo realice llamadas a herramientas, usamos .bind_tools** para brindarle al modelo de lenguaje conocimiento de estas herramientas:

In [ ]:
llm_with_tools = chatModel.bind_tools(tools)

## Crear el agente
* Usaremos LangGraph para construir el agente.
* **Tenga en cuenta que a continuación pasamos el modelo de chat original, no el llm_with_tools que creamos más tarde**. Esto se debe a que create_tool_calling_executor llamará a .bind_tools por nosotros en forma interna.

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(chatModel, tools)

## Ejecutar el agente
* Primero, intentémoslo con .invoke():

In [ ]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="¿Dónde se jugó la Euro 2024?")]})

response["messages"]

*Ahora probémoslo con .stream():

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="When and where will it be the 2024 Eurocup final match?")]}
):
    print(chunk)
    print("----")

## Agregar memoria
* Agregar memoria en LangGraph es muy similar a lo que hicimos con LangChain.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

* Creemos nuestro nuevo agente con memoria y establezcamos un thread_id para que el agente pueda crear una memoria para cada sesión como lo hicimos con nuestra aplicación RAG conversacional anterior:

In [ ]:
agent_executor = create_react_agent(chatModel, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "001"}}

* Probemos ahora este nuevo agente con .stream():

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="¿Quién ganó la Euro 2024?")]}, config
):
    print(chunk)
    print("----")

* Cambiemos el thread_id y veamos qué sucede:

In [ ]:
config = {"configurable": {"thread_id": "002"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="¿Sobre que jugador estamos hablando?")]}, config
):
    print(chunk)
    print("----")

* Como puedes observar al cambiar el threat_id iniciamos una memoria de conversación diferente, por lo que nuestra app no ​​recuerda la interacción anterior y en lugar de darnos la respuesta correcta decidió alucinar.